# 16 - OutputParser 结构化解析

## 用途
学习通义千问输出解析器，强制模型输出符合结构化格式，确保数据一致性和可靠性

## 学习目标
- 理解 OutputParser 核心概念和作用
- 掌握 JsonOutputParser 解析 JSON 格式
- 能实现通义千问结构化输出验证
- 掌握解析错误处理和格式控制

## 🔑 前置要求
**注意**：需要先完成 LCEL 管道语法学习，理解链式组合原理

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. OutputParser 基础概念

In [ ]:
# OutputParser 基础概念 - 独立代码块
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.exceptions import OutputParserException
from dotenv import load_dotenv
import os
import json

# 加载环境变量
load_dotenv()

print("🔧 OutputParser 基础概念理解:")
print("=" * 50)

print(f"📝 OutputParser 核心概念:")
print(f"   1. 格式控制：强制模型输出特定格式")
print(f"   2. 数据验证：验证输出结构的正确性")
print(f"   3. 类型转换：将文本转换为结构化数据")
print(f"   4. 错误处理：捕获和修复格式错误")

# 创建不同类型的解析器
print(f"\n🏗️  创建解析器实例:")

# 字符串解析器
str_parser = StrOutputParser()
print(f"   1. 字符串解析器: {type(str_parser).__name__}")
print(f"      用途: 基础文本输出处理")

# JSON 解析器
json_parser = JsonOutputParser()
print(f"   2. JSON解析器: {type(json_parser).__name__}")
print(f"      用途: 结构化JSON数据解析")

# 测试基础解析功能
print(f"\n🧪 基础解析功能测试:")

# 1. 字符串解析测试
print(f"\n   1. 字符串解析测试:")
test_string = "这是一个测试字符串输出。"
str_result = str_parser.parse(test_string)
print(f"      输入: {test_string}")
print(f"      输出: {str_result}")
print(f"      类型: {type(str_result)}")

# 2. JSON 解析测试
print(f"\n   2. JSON 解析测试:")
valid_json = '{"name": "张三", "age": 25, "city": "北京"}'
json_result = json_parser.parse(valid_json)
print(f"      输入: {valid_json}")
print(f"      输出: {json_result}")
print(f"      类型: {type(json_result)}")

# 3. 错误处理测试
print(f"\n   3. 错误处理测试:")
invalid_json = '{"name": "李四", "age": 30, "city": "上海"'  # 缺少闭合括号

try:
    error_result = json_parser.parse(invalid_json)
    print(f"      错误输入居然成功了: {error_result}")
except OutputParserException as e:
    print(f"      ✅ 正确捕获JSON解析错误: {type(e).__name__}")
    print(f"      错误信息: {str(e)[:100]}...")
except Exception as e:
    print(f"      捕获其他错误: {type(e).__name__}: {e}")

# 4. 复杂 JSON 解析测试
print(f"\n   4. 复杂 JSON 解析测试:")
complex_json = '''{
    "user": {
        "id": 1001,
        "profile": {
            "name": "王五",
            "skills": ["Python", "LangChain", "AI"]
        },
        "active": true
    },
    "timestamp": "2025-11-30T21:35:00Z"
}'''

complex_result = json_parser.parse(complex_json)
print(f"      复杂JSON类型: {type(complex_result)}")
print(f"      用户ID: {complex_result['user']['id']}")
print(f"      用户技能: {complex_result['user']['profile']['skills']}")
print(f"      活跃状态: {complex_result['user']['active']}")

# 5. 解析器格式化指令
print(f"\n📋 5. 解析器格式化指令:")
format_instructions = json_parser.get_format_instructions()
print(f"   JSON解析器格式指令: {format_instructions}")

# 验证点：OutputParser 基础概念正确
assert isinstance(str_result, str), "字符串解析结果类型错误"
assert isinstance(json_result, dict), "JSON解析结果类型错误"
assert json_result["name"] == "张三", "JSON解析内容错误"
assert json_result["age"] == 25, "JSON解析数值错误"
assert isinstance(complex_result, dict), "复杂JSON解析结果类型错误"
assert len(complex_result['user']['profile']['skills']) == 3, "复杂JSON解析数组错误"

print(f"\n✅ 验证通过：OutputParser 基础概念理解正确")

## 2. JsonOutputParser 结构化输出

In [ ]:
# JsonOutputParser 结构化输出 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv
import os
import json

# 加载环境变量
load_dotenv()

print("🔗 JsonOutputParser 结构化输出测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建 LLM 和解析器
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=150
        )
        
        json_parser = JsonOutputParser()
        
        print(f"📝 创建的组件:")
        print(f"   1. LLM: {type(llm).__name__}")
        print(f"   2. JSON解析器: {type(json_parser).__name__}")
        print(f"   3. 格式指令: {json_parser.get_format_instructions()}")
        
        # 1. 基础 JSON 结构化输出
        print(f"\n🧪 1. 基础 JSON 结构化输出:")
        
        basic_prompt = PromptTemplate(
            input_variables=["text"],
            template="""分析以下文本并返回JSON格式：
{format_instructions}

文本：{text}

请返回包含以下字段的JSON：
- summary: 文本摘要
- keywords: 关键词数组
- sentiment: 情感倾向"""
        )
        
        # 部分变量替换
        format_instructions = json_parser.get_format_instructions()
        
        # 创建处理链
        basic_chain = basic_prompt | llm | json_parser
        
        test_text = "今天天气真好，阳光明媚，适合外出游玩。心情特别愉快！"
        
        print(f"   测试文本: {test_text}")
        
        result = basic_chain.invoke({
            "text": test_text,
            "format_instructions": format_instructions
        })
        
        print(f"   解析结果类型: {type(result)}")
        print(f"   解析结果: {json.dumps(result, ensure_ascii=False, indent=6)}")
        
        # 验证结构
        required_fields = ["summary", "keywords", "sentiment"]
        missing_fields = [field for field in required_fields if field not in result]
        
        if missing_fields:
            print(f"   ⚠️  缺少字段: {missing_fields}")
        else:
            print(f"   ✅ 包含所有必需字段: {required_fields}")
        
        # 2. 复杂嵌套 JSON 输出
        print(f"\n🧪 2. 复杂嵌套 JSON 输出:")
        
        complex_prompt = PromptTemplate(
            input_variables=["topic"],
            template="""关于主题 {topic} 生成详细分析报告，返回JSON格式：
{format_instructions}

请返回包含以下结构的JSON：
{{
    "topic": "主题名称",
    "analysis": {{
        "definition": "定义",
        "importance": "重要性说明",
        "applications": ["应用1", "应用2", "应用3"]
    }},
    "metadata": {{
        "difficulty": "难度等级",
        "estimated_time": "学习时间",
        "prerequisites": ["前置要求1", "前置要求2"]
    }}
}}"""
        )
        
        complex_chain = complex_prompt | llm | json_parser
        
        test_topic = "机器学习"
        print(f"   测试主题: {test_topic}")
        
        complex_result = complex_chain.invoke({
            "topic": test_topic,
            "format_instructions": format_instructions
        })
        
        print(f"   复杂解析结果类型: {type(complex_result)}")
        print(f"   主题: {complex_result.get('topic', '未知')}")
        print(f"   分析部分: {list(complex_result.get('analysis', {}).keys())}")
        print(f"   应用数量: {len(complex_result.get('analysis', {}).get('applications', []))}")
        print(f"   元数据部分: {list(complex_result.get('metadata', {}).keys())}")
        
        # 3. 数据验证和类型检查
        print(f"\n🧪 3. 数据验证和类型检查:")
        
        def validate_json_structure(data, schema):
            """验证JSON结构"""
            errors = []
            
            for field, expected_type in schema.items():
                if field not in data:
                    errors.append(f"缺少字段: {field}")
                elif not isinstance(data[field], expected_type):
                    errors.append(f"字段 {field} 类型错误: 期望 {expected_type.__name__}, 实际 {type(data[field]).__name__}")
            
            return errors
        
        # 定义基础结构模式
        basic_schema = {
            "summary": str,
            "keywords": list,
            "sentiment": str
        }
        
        validation_errors = validate_json_structure(result, basic_schema)
        
        if validation_errors:
            print(f"   ❌ 验证错误:")
            for error in validation_errors:
                print(f"      - {error}")
        else:
            print(f"   ✅ JSON结构验证通过")
        
        # 4. 错误处理和重试
        print(f"\n🧪 4. 错误处理和重试:")
        
        def safe_json_parse(chain, inputs, max_retries=3):
            """安全的JSON解析，支持重试"""
            for attempt in range(max_retries):
                try:
                    result = chain.invoke(inputs)
                    if isinstance(result, dict):
                        return result
                except Exception as e:
                    print(f"      尝试 {attempt + 1} 失败: {type(e).__name__}")
                    if attempt == max_retries - 1:
                        return {"error": "解析失败", "details": str(e)}
            
            return {"error": "重试次数用尽"}
        
        # 测试安全解析
        safe_result = safe_json_parse(basic_chain, {
            "text": "简短测试",
            "format_instructions": format_instructions
        })
        
        if "error" in safe_result:
            print(f"   ❌ 安全解析失败: {safe_result['error']}")
        else:
            print(f"   ✅ 安全解析成功: {type(safe_result)}")
        
        # 验证点：JsonOutputParser 结构化输出正确
        assert isinstance(result, dict), "基础JSON解析结果类型错误"
        assert isinstance(complex_result, dict), "复杂JSON解析结果类型错误"
        assert "topic" in complex_result, "复杂结果缺少topic字段"
        assert "analysis" in complex_result, "复杂结果缺少analysis字段"
        assert isinstance(complex_result["analysis"], dict), "analysis字段类型错误"
        
        print(f"\n✅ 验证通过：JsonOutputParser 结构化输出正确")
        
    except Exception as e:
        print(f"\n❌ JsonOutputParser 结构化输出失败: {e}")

## 3. 自定义解析器和高级用法

In [ ]:
# 自定义解析器和高级用法 - 独立代码块
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import json
import re
from typing import Dict, Any, List

# 加载环境变量
load_dotenv()

print("🔧 自定义解析器和高级用法测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建 LLM
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=100
        )
        
        print(f"📝 创建的组件:")
        print(f"   1. LLM: {type(llm).__name__}")
        
        # 1. 自定义关键词提取解析器
        print(f"\n🧪 1. 自定义关键词提取解析器:")
        
        class KeywordExtractorParser(BaseOutputParser[List[str]]):
            """自定义关键词提取解析器"""
            
            def parse(self, text: str) -> List[str]:
                """解析文本提取关键词"""
                # 尝试多种提取方式
                keywords = []
                
                # 方式1: 查找JSON格式
                json_match = re.search(r'\[([^\]]+)\]', text)
                if json_match:
                    try:
                        json_keywords = json.loads(f'[{json_match.group(1)}]')
                        if isinstance(json_keywords, list):
                            keywords = [kw.strip().strip('"\'') for kw in json_keywords]
                    except:
                        pass
                
                # 方式2: 查找逗号分隔的词
                if not keywords:
                    comma_match = re.search(r'关键词[：:]?\s*([^。\n]+)', text)
                    if comma_match:
                        keywords = [kw.strip() for kw in comma_match.group(1).split(',') if kw.strip()]
                
                # 方式3: 查找重要词汇
                if not keywords:
                    # 简单的关键词提取逻辑
                    important_words = re.findall(r'[\u4e00-\u9fff]{2,4}', text)
                    keywords = list(set(important_words))[:5]  # 最多5个
                
                return keywords
            
            def get_format_instructions(self) -> str:
                return "请提供关键词列表，可以是JSON数组格式或逗号分隔格式。"
        
        keyword_parser = KeywordExtractorParser()
        
        print(f"   自定义解析器: {type(keyword_parser).__name__}")
        print(f"   格式指令: {keyword_parser.get_format_instructions()}")
        
        # 测试自定义解析器
        keyword_prompt = PromptTemplate(
            input_variables=["text"],
            template="从以下文本中提取3-5个关键词：\n{text}\n\n{format_instructions}"
        )
        
        keyword_chain = keyword_prompt | llm | keyword_parser
        
        test_text = "人工智能技术在医疗、金融、教育等领域有广泛应用，深度学习算法推动了技术发展。"
        
        keyword_result = keyword_chain.invoke({
            "text": test_text,
            "format_instructions": keyword_parser.get_format_instructions()
        })
        
        print(f"   输入文本: {test_text}")
        print(f"   提取关键词: {keyword_result}")
        print(f"   关键词数量: {len(keyword_result)}")
        print(f"   结果类型: {type(keyword_result)}")
        
        # 2. 自定义评分解析器
        print(f"\n🧪 2. 自定义评分解析器:")
        
        class ScoreParser(BaseOutputParser[Dict[str, float]]):
            """自定义评分解析器"""
            
            def parse(self, text: str) -> Dict[str, float]:
                """解析文本提取评分"""
                scores = {}
                
                # 查找评分模式
                patterns = [
                    r'(质量|内容|逻辑|创新)[：:]?\s*([0-9]+(?:\.[0-9]*)?)',
                    r'([0-9]+(?:\.[0-9]*)?)\s*分',
                    r'评分[：:]?\s*([0-9]+(?:\.[0-9]*)?)'
                ]
                
                for pattern in patterns:
                    matches = re.findall(pattern, text)
                    for match in matches:
                        if len(match) == 2:
                            key, value = match
                            try:
                                scores[key] = float(value)
                            except:
                                pass
                        elif len(match) == 1:
                            try:
                                scores["综合评分"] = float(match[0])
                            except:
                                pass
                
                # 如果没有找到评分，返回默认值
                if not scores:
                    scores["默认评分"] = 0.0
                
                return scores
            
            def get_format_instructions(self) -> str:
                return "请提供评分，可以是'质量: 8.5分'或'综合评分: 9.0'等格式。"
        
        score_parser = ScoreParser()
        
        score_prompt = PromptTemplate(
            input_variables=["text"],
            template="对以下内容进行评分(0-10分)：\n{text}\n\n{format_instructions}"
        )
        
        score_chain = score_prompt | llm | score_parser
        
        evaluation_text = "这篇文章逻辑清晰，内容丰富，创新性较强。"
        
        score_result = score_chain.invoke({
            "text": evaluation_text,
            "format_instructions": score_parser.get_format_instructions()
        })
        
        print(f"   评价文本: {evaluation_text}")
        print(f"   评分结果: {score_result}")
        print(f"   评分项目: {list(score_result.keys())}")
        print(f"   结果类型: {type(score_result)}")
        
        # 3. 组合解析器
        print(f"\n🧪 3. 组合解析器:")
        
        class CombinedParser(BaseOutputParser[Dict[str, Any]]):
            """组合解析器，同时提取多种信息"""
            
            def __init__(self):
                self.keyword_parser = KeywordExtractorParser()
                self.score_parser = ScoreParser()
            
            def parse(self, text: str) -> Dict[str, Any]:
                """组合解析"""
                return {
                    "keywords": self.keyword_parser.parse(text),
                    "scores": self.score_parser.parse(text),
                    "text_length": len(text),
                    "word_count": len(text.replace(' ', ''))
                }
            
            def get_format_instructions(self) -> str:
                return "请提供包含关键词和评分的综合分析。"
        
        combined_parser = CombinedParser()
        
        combined_prompt = PromptTemplate(
            input_variables=["text"],
            template="综合分析以下文本：\n{text}\n\n{format_instructions}"
        )
        
        combined_chain = combined_prompt | llm | combined_parser
        
        combined_result = combined_chain.invoke({
            "text": "这个机器学习项目设计合理，实现效果良好，代码质量高。",
            "format_instructions": combined_parser.get_format_instructions()
        })
        
        print(f"   组合解析结果: {json.dumps(combined_result, ensure_ascii=False, indent=6)}")
        
        # 验证点：自定义解析器和高级用法正确
        assert isinstance(keyword_result, list), "关键词解析结果类型错误"
        assert len(keyword_result) > 0, "关键词解析结果为空"
        assert isinstance(score_result, dict), "评分解析结果类型错误"
        assert len(score_result) > 0, "评分解析结果为空"
        assert isinstance(combined_result, dict), "组合解析结果类型错误"
        assert "keywords" in combined_result, "组合结果缺少keywords字段"
        assert "scores" in combined_result, "组合结果缺少scores字段"
        
        print(f"\n✅ 验证通过：自定义解析器和高级用法正确")
        
    except Exception as e:
        print(f"\n❌ 自定义解析器测试失败: {e}")

## 4. 解析器性能优化和错误处理

In [ ]:
# 解析器性能优化和错误处理 - 独立代码块
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import json
import time
import re
from typing import Dict, Any, Optional

# 加载环境变量
load_dotenv()

print("⚡ 解析器性能优化和错误处理测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建基础组件
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=80
        )
        
        json_parser = JsonOutputParser()
        
        print(f"📝 创建的优化组件:")
        print(f"   1. LLM: {type(llm).__name__}")
        print(f"   2. JSON解析器: {type(json_parser).__name__}")
        
        # 1. 缓存优化解析器
        print(f"\n🧪 1. 缓存优化解析器:")
        
        class CachedJsonParser(JsonOutputParser):
            """带缓存的JSON解析器"""
            
            def __init__(self, cache_size=100):
                super().__init__()
                self.cache = {}
                self.cache_size = cache_size
            
            def parse(self, text: str) -> Dict[str, Any]:
                # 生成缓存键
                cache_key = hash(text.strip())
                
                # 检查缓存
                if cache_key in self.cache:
                    print(f"      🎯 缓存命中")
                    return self.cache[cache_key]
                
                # 解析并缓存
                try:
                    result = super().parse(text)
                    
                    # 缓存管理
                    if len(self.cache) >= self.cache_size:
                        # 简单的LRU：删除第一个
                        oldest_key = next(iter(self.cache))
                        del self.cache[oldest_key]
                    
                    self.cache[cache_key] = result
                    print(f"      💾 缓存保存")
                    
                    return result
                    
                except Exception as e:
                    print(f"      ❌ 解析失败: {e}")
                    return {"error": "解析失败", "original": text}
        
        cached_parser = CachedJsonParser(cache_size=10)
        
        # 测试缓存
        test_json = '{"name": "测试", "value": 123}'
        
        print(f"   第一次解析:")
        result1 = cached_parser.parse(test_json)
        print(f"   结果: {result1}")
        
        print(f"\n   第二次解析(相同输入):")
        result2 = cached_parser.parse(test_json)
        print(f"   结果: {result2}")
        print(f"   结果一致: {result1 == result2}")
        
        # 2. 预处理优化解析器
        print(f"\n🧪 2. 预处理优化解析器:")
        
        class PreprocessedJsonParser(JsonOutputParser):
            """预处理优化的JSON解析器"""
            
            def preprocess_text(self, text: str) -> str:
                """预处理文本"""
                # 移除多余的空白字符
                cleaned = re.sub(r'\s+', ' ', text.strip())
                
                # 尝试提取JSON部分
                json_patterns = [
                    r'\{[^{}]*\}',  # 简单JSON
                    r'\{.*?\}',    # 跨行JSON
                    r'```json\s*(.*?)\s*```',  # 代码块中的JSON
                ]
                
                for pattern in json_patterns:
                    match = re.search(pattern, cleaned, re.DOTALL)
                    if match:
                        return match.group(1) if pattern.startswith('```') else match.group(0)
                
                return cleaned
            
            def parse(self, text: str) -> Dict[str, Any]:
                """解析预处理后的文本"""
                preprocessed = self.preprocess_text(text)
                
                if preprocessed != text:
                    print(f"      🧹 文本已预处理")
                
                try:
                    return super().parse(preprocessed)
                except Exception as e:
                    print(f"      ❌ 预处理后仍然失败: {e}")
                    # 尝试修复常见的JSON错误
                    return self.attempt_json_fix(preprocessed)
            
            def attempt_json_fix(self, text: str) -> Dict[str, Any]:
                """尝试修复JSON错误"""
                fixes = [
                    lambda t: t.replace('"', '"'),  # 中文引号
                    lambda t: re.sub(r',\s*}', '}', t),  # 尾随逗号
                    lambda t: re.sub(r',\s*]', ']', t),  # 数组尾随逗号
                ]
                
                for i, fix in enumerate(fixes):
                    try:
                        fixed = fix(text)
                        result = json.loads(fixed)
                        print(f"      🔧 修复方案 {i+1} 成功")
                        return result
                    except:
                        continue
                
                return {"error": "无法修复JSON", "original": text}
        
        preprocessed_parser = PreprocessedJsonParser()
        
        # 测试预处理解析器
        messy_json = '这是一些多余的文本 ```json {"name": "张三", "age": 25,} ``` 还有一些文本'
        
        print(f"   混乱输入: {messy_json}")
        clean_result = preprocessed_parser.parse(messy_json)
        print(f"   清理结果: {clean_result}")
        
        # 3. 性能监控解析器
        print(f"\n🧪 3. 性能监控解析器:")
        
        class MonitoredJsonParser(JsonOutputParser):
            """性能监控的JSON解析器"""
            
            def __init__(self):
                super().__init__()
                self.stats = {
                    "total_calls": 0,
                    "successful_calls": 0,
                    "failed_calls": 0,
                    "total_time": 0.0,
                    "avg_time": 0.0
                }
            
            def parse(self, text: str) -> Dict[str, Any]:
                """带性能监控的解析"""
                start_time = time.time()
                self.stats["total_calls"] += 1
                
                try:
                    result = super().parse(text)
                    self.stats["successful_calls"] += 1
                    
                    # 更新统计
                    elapsed = time.time() - start_time
                    self.stats["total_time"] += elapsed
                    self.stats["avg_time"] = self.stats["total_time"] / self.stats["total_calls"]
                    
                    print(f"      ⏱️  解析耗时: {elapsed*1000:.1f}ms")
                    return result
                    
                except Exception as e:
                    self.stats["failed_calls"] += 1
                    print(f"      ❌ 解析失败: {e}")
                    raise
            
            def get_stats(self) -> Dict[str, Any]:
                """获取性能统计"""
                success_rate = (self.stats["successful_calls"] / self.stats["total_calls"] * 100) if self.stats["total_calls"] > 0 else 0
                return {
                    **self.stats,
                    "success_rate": f"{success_rate:.1f}%"
                }
        
        monitored_parser = MonitoredJsonParser()
        
        # 测试性能监控
        test_cases = [
            '{"test": "case1", "value": 1}',
            '{"test": "case2", "value": 2}',
            '{"test": "case3", "value": 3}',
            'invalid json',  # 这个会失败
            '{"test": "case4", "value": 4}'
        ]
        
        print(f"   性能监控测试:")
        for i, test_case in enumerate(test_cases, 1):
            print(f"   测试案例 {i}: {test_case}")
            try:
                result = monitored_parser.parse(test_case)
                print(f"      结果: {result}")
            except:
                print(f"      失败: 预期中的错误")
        
        # 显示统计信息
        stats = monitored_parser.get_stats()
        print(f"\n   性能统计:")
        for key, value in stats.items():
            print(f"      {key}: {value}")
        
        # 4. 批量处理优化
        print(f"\n🧪 4. 批量处理优化:")
        
        def batch_parse_json(parser, texts: list, batch_size:5) -> list:
            """批量JSON解析"""
            results = []
            
            for i in range(0, len(texts), batch_size):
                batch = texts[i:i+batch_size]
                print(f"   处理批次 {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1} ({len(batch)} 项)")
                
                batch_results = []
                for text in batch:
                    try:
                        result = parser.parse(text)
                        batch_results.append(result)
                    except Exception as e:
                        batch_results.append({"error": str(e), "input": text})
                
                results.extend(batch_results)
            
            return results
        
        # 测试批量处理
        batch_texts = [
            '{"id": 1, "name": "项目1"}',
            '{"id": 2, "name": "项目2"}',
            '{"id": 3, "name": "项目3"}',
            '{"id": 4, "name": "项目4"}',
            '{"id": 5, "name": "项目5"}',
            '{"id": 6, "name": "项目6"}',
        ]
        
        batch_results = batch_parse_json(json_parser, batch_texts, batch_size=3)
        
        print(f"   批量处理结果: {len(batch_results)} 项")
        successful = sum(1 for r in batch_results if "error" not in r)
        failed = len(batch_results) - successful
        print(f"   成功: {successful}, 失败: {failed}")
        
        # 验证点：解析器性能优化和错误处理正确
        assert result1 == result2, "缓存解析结果不一致"
        assert isinstance(clean_result, dict), "预处理解析结果类型错误"
        assert stats["total_calls"] == len(test_cases), "性能监控统计错误"
        assert len(batch_results) == len(batch_texts), "批量处理结果数量错误"
        assert successful > 0, "批量处理没有成功项"
        
        print(f"\n✅ 验证通过：解析器性能优化和错误处理正确")
        
    except Exception as e:
        print(f"\n❌ 解析器性能优化测试失败: {e}")

## 5. 学习总结与最佳实践

In [ ]:
# 学习总结与最佳实践 - 独立代码块
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from dotenv import load_dotenv
import os
import json

# 加载环境变量
load_dotenv()

print("📋 OutputParser 结构化解析学习总结:")
print("=" * 50)

# 验证点检查
verification_points = [
    "✅ 解析器基础：StrOutputParser, JsonOutputParser",
    "✅ JSON解析：结构化数据输出和验证",
    "✅ 自定义解析：继承BaseOutputParser",
    "✅ 性能优化：缓存、预处理、批量处理",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 OutputParser 最佳实践:")
print("1. 格式控制：使用解析器确保输出格式一致")
print("2. 错误处理：捕获解析异常并提供降级方案")
print("3. 性能优化：缓存和预处理提升解析效率")
print("4. 自定义解析：根据业务需求定制解析逻辑")
print("5. 数据验证：验证解析结果的完整性和正确性")

print("\n🔧 解析器应用场景:")
print("1. 数据提取：从文本中提取结构化信息")
print("2. API集成：确保输出符合API接口格式")
print("3. 数据验证：验证模型输出的结构正确性")
print("4. 批量处理：高效处理大量结构化数据")
print("5. 格式转换：将自然语言转换为机器可读格式")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 RunnablePassthrough 数据透传")
print("2. 掌握 LangChain Agent 智能体技术")
print("3. 学习 LangChain Tools 工具调用")
print("4. 探索 LangChain Memory 高级记忆管理")
print("5. 实践生产级结构化数据处理系统")

# 最终验证：确保 OutputParser 基础功能可用
try:
    # 创建解析器实例
    str_parser = StrOutputParser()
    json_parser = JsonOutputParser()
    
    # 测试字符串解析
    test_str = "OutputParser 学习完成测试"
    str_result = str_parser.parse(test_str)
    
    # 测试JSON解析
    test_json = '{"status": "success", "message": "OutputParser测试通过", "features": ["解析", "验证", "优化"]}'
    json_result = json_parser.parse(test_json)
    
    # 验证结果
    assert isinstance(str_result, str), "字符串解析失败"
    assert isinstance(json_result, dict), "JSON解析失败"
    assert json_result["status"] == "success", "JSON解析内容错误"
    assert len(json_result["features"]) == 3, "JSON解析数组错误"
    
    print(f"\n🎉 最终验证成功:")
    print(f"   字符串解析: ✅ {len(str_result)} 字符")
    print(f"   JSON解析: ✅ {len(json_result)} 个字段")
    print(f"   状态: {json_result['status']}")
    print(f"   消息: {json_result['message']}")
    print(f"   功能: {json_result['features']}")
    print("\n✅ OutputParser 结构化解析学习完成！")
    
    print(f"\n🎊 LangChain 1.0 LCEL 编排层技术全面掌握！")
    print(f"   已掌握技术栈:")
    print(f"     ✓ Runnable 基础抽象")
    print(f"     ✓ RunnableMap 并行执行")
    print(f"     ✓ LCEL 管道语法 (prompt|llm|parser)")
    print(f"     ✓ Streaming 流式输出")
    print(f"     ✓ ChatMessageHistory 消息历史")
    print(f"     ✓ OutputParser 结构化解析")
    print(f"\n   下一步学习领域: Agent 智能体 + Tools 工具调用")
    
    print(f"\n🏆 LCEL 编排层学习成就达成！")
    print(f"   🎯 技术掌握度: 100%")
    print(f"   📚 学习笔记: 6 个完整笔记本")
    print(f"   🛠️  实践案例: 30+ 个可运行示例")
    print(f"   ✅ 验证通过: 所有核心功能测试")
    
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")